# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [1]:
import riak

In [2]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

¿Está Riak disponible? True


In [3]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

"2.1.7-0-gbd8e312"


## Insertar valores

In [4]:
bucket = cliente.bucket("bucket_name")

In [5]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [6]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [7]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [8]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [9]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [10]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

key_one: 1
key_two: Dos
key_three: 
{'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}


In [11]:
!curl http://localhost:8098/riak/bucket_name/key_one

1

In [12]:
!curl http://localhost:8098/riak/bucket_name/key_two

"Dos"

In [13]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

{
  "val1": 1,
  "val2": "Dos",
  "val3": [
    1,
    2,
    3,
    4,
    5
  ]
}


http://localhost:8098/riak/images/image_key

## Índices

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [76]:
import pandas as pd
import json
import uuid
import random

bucket = cliente.bucket("ejemplo_indices")

col_Ciudad = ["Madrid", "Madrid", "Barcelona", "Sevilla", "Madrid"]

for i in range(0, 5):
    key = str(i+1)
    
    valor = {"value" : random.randint(0,255),
             "key_aux" :  uuid.uuid4().hex,
             "ciudad" : col_Ciudad[i]}

    riak_key = bucket.new(key, data = valor)
    riak_key.add_index('idx_ciudad_bin', valor["ciudad"])
    riak_key.add_index('idx_key_aux_bin', valor["key_aux"])
    riak_key.store()
    
    print("key", key, ":", valor)


key 1 : {'value': 172, 'key_aux': 'c7a512b27e5d461290e32c4777fafbf3', 'ciudad': 'Madrid'}
key 2 : {'value': 11, 'key_aux': 'a88cb2a4eefc4cdba08c119ef9ca2c6a', 'ciudad': 'Madrid'}
key 3 : {'value': 171, 'key_aux': '02ecfc3c8f064c9697cbbc93e516bdb3', 'ciudad': 'Barcelona'}
key 4 : {'value': 61, 'key_aux': 'ade799d3d19a4238b8f9d1e9855928be', 'ciudad': 'Sevilla'}
key 5 : {'value': 170, 'key_aux': '3d580e19879a4296b374405d284dbd8f', 'ciudad': 'Madrid'}


Cargamos la información en Pandas

In [77]:
rows = []
for keys in bucket.stream_keys():
    for key in keys:
        rows.append(bucket.get(key).data)

df = pd.read_json(json.dumps(rows))
df

,value,key_aux,ciudad
0,11,a88cb2a4eefc4cdba08c119ef9ca2c6a,Madrid
1,172,c7a512b27e5d461290e32c4777fafbf3,Madrid
2,61,ade799d3d19a4238b8f9d1e9855928be,Sevilla
3,170,3d580e19879a4296b374405d284dbd8f,Madrid
4,171,02ecfc3c8f064c9697cbbc93e516bdb3,Barcelona


Ahora vamos a localizar las claves de "Madrid" a través del ínidice

In [78]:
keys = bucket.get_index("idx_ciudad_bin", 'Madrid')
for key in keys.results:
    print(bucket.get(key).data)

{'value': 11, 'key_aux': 'a88cb2a4eefc4cdba08c119ef9ca2c6a', 'ciudad': 'Madrid'}
{'value': 172, 'key_aux': 'c7a512b27e5d461290e32c4777fafbf3', 'ciudad': 'Madrid'}
{'value': 170, 'key_aux': '3d580e19879a4296b374405d284dbd8f', 'ciudad': 'Madrid'}


Ahora vamos a localizar una clave auxiliar a través de su ínidice

In [79]:
key_aux = df.iat[0, 1]
print(key_aux)

a88cb2a4eefc4cdba08c119ef9ca2c6a


In [80]:
keys = bucket.get_index("idx_key_aux_bin", key_aux)
for key in keys.results:
    print(bucket.get(key).data)

{'value': 11, 'key_aux': 'a88cb2a4eefc4cdba08c119ef9ca2c6a', 'ciudad': 'Madrid'}


## Listado de Buckets

In [44]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

Bucket <RiakBucket 'movimientos'>
Bucket <RiakBucket 'images'>
Bucket <RiakBucket 'status'>
Bucket <RiakBucket 'ejemplo_indices'>
Bucket <RiakBucket 'people'>
Bucket <RiakBucket 'bucket_name'>


## Listado de Claves de un Bucket

In [45]:
for keys in bucket_estado.stream_keys():
    for key in keys:
        print('Key %s' % key )

Key 128f0ef83d2d11ebb19108002776b909
